<a href="https://colab.research.google.com/github/yeeeeeun/GEC_by_edit_spans/blob/main/GEC_by_edit_spans_date_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## 데이터 전처리. LLM 학습.

In [ ]:
!pip install soynlp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 10.1 MB/s eta 0:00:00


In [ ]:
from soynlp.tokenizer import RegexTokenizer
from difflib import SequenceMatcher

tokenizer = RegexTokenizer()

def extract_correction_ranges(input_text, corrected_text):
    ranges = []
    matcher = SequenceMatcher(None, input_text, corrected_text)

    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag in {"replace", "delete", "insert"}:
            ranges.append((i1, i2, corrected_text[j1:j2]))
    return ranges

def process_data(input_text, gold_text):
    tokens_input = tokenizer.tokenize(input_text)
    tokens_gold = tokenizer.tokenize(gold_text)

    ranges = extract_correction_ranges(input_text, gold_text)

    ranges_str = ", ".join(f"({start}, {end}, '{replace}')" for start, end, replace in ranges)
    return {"input": input_text, "output": ranges_str}

# 데이터셋 예제
data = [
    {"input": "어쩌다 가게되써", "gold": "어쩌다 가게됐어?"},
    {"input": "돌아댕겨야지", "gold": "돌아다녀야지."},
    {"input": "오오 어디?", "gold": "오오, 어디?"},
    {"input": "ㅋㅋㅋㅋ헐 대박 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 어때?", "gold": "ㅋㅋㅋㅋ 헐, 대박 ㅋㅋㅋㅋㅋ. 어때?"}
]

# 데이터 처리
output_data = [process_data(item["input"], item["gold"]) for item in data]

# 결과 출력
for entry in output_data:
    print(entry)


{'input': '어쩌다 가게되써', 'output': "(6, 8, '됐어?')"}
{'input': '돌아댕겨야지', 'output': "(2, 4, '다녀'), (6, 6, '.')"}
{'input': '오오 어디?', 'output': "(2, 2, ',')"}
{'input': 'ㅋㅋㅋㅋ헐 대박 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 어때?', 'output': "(4, 4, ' '), (5, 5, ','), (14, 18, '.')"}


/usr/local/lib/python3.10/dist-packages/soynlp/tokenizer/_tokenizer.py:19: FutureWarning: Possible nested set at position 13
  ('english & latin', re.compile(u"[a-zA-ZÀ-ÿ]+[[`']?s]*|[a-zA-ZÀ-ÿ]+", re.UNICODE))


In [ ]:
# time.time() - test_step

In [ ]:
import json
from soynlp.tokenizer import RegexTokenizer
from difflib import SequenceMatcher

tokenizer = RegexTokenizer()

def extract_correction_ranges(input_text, corrected_text):
    ranges = []
    matcher = SequenceMatcher(None, input_text, corrected_text)

    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag in {"replace", "delete", "insert"}:
            ranges.append((i1, i2, corrected_text[j1:j2]))
    return ranges

def process_data(input_text, gold_text):
    ranges = extract_correction_ranges(input_text, gold_text)

    ranges_str = ", ".join(f"({start}, {end}, '{replace}')" for start, end, replace in ranges)
    return {"input": input_text, "output": ranges_str}

def process_json(input_json):
    output_data = []

    for document in input_json["document"]:
        for utterance in document["utterance"]:
            input_text = utterance["original_form"]
            gold_text = utterance["corrected_form"]
            processed = process_data(input_text, gold_text)
            output_data.append(processed)

    return output_data

input_file = '/content/preprocessed_dataset.json'
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

output_data = process_json(data)

for entry in output_data:
    print(entry)

output_file = 'processed_data.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, ensure_ascii=False, indent=4)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
{'input': '얌 찜갈비 역근처였다나', 'output': "(0, 1, '야,'), (7, 7, ' '), (12, 12, '?')"}
{'input': '다음날 집갈떄 먹고 헤어지면 되겠네!', 'output': "(2, 2, ' '), (5, 5, ' '), (6, 7, ' 때')"}
{'input': '정확히는 김광석길근처란다', 'output': "(8, 8, ' '), (9, 9, ' '), (13, 13, '.')"}
{'input': '오 그것도 괜춘', 'output': "(1, 1, ','), (7, 8, '찮아.')"}
{'input': '그럼 첫쨋날 저녁은 이월드랑 서문시장에서?', 'output': "(4, 5, '째 ')"}
{'input': '다음날다음날! 그런데 애들이 바로 집가고 싶어할까?', 'output': "(2, 2, ' '), (3, 3, ', '), (5, 5, ' '), (20, 20, ' '), (25, 25, ' ')"}
{'input': '다음날 한 7시 귀가하자', 'output': "(2, 2, ' '), (13, 13, '.')"}
{'input': '응 첫째날 식사는 막창 이월드 서문시장!', 'output': "(1, 1, ','), (4, 4, ' '), (12, 12, ',')"}
{'input': '막창 막창거리가서 먹어 아님 딴데서 먹어', 'output': "(0, 0, '막창, '), (3, 5, ''), (7, 7, ' '), (12, 12, ','), (17, 17, ' '), (22, 22, '?')"}
{'input': '나 대구 중구 맛집 알아왔느데', 'output': "(4, 4, ','), (13, 13, ' '), (14, 15, '는')"}
{'input': '좀 많음', 'output': "(4, 4, '.')"}
{'input': '오키오키 좋음 가서 가까운데서 ! 먹먹자', 'output': "(4, 

In [ ]:
import json
import random
import re

def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

def filter_data(data):
    pattern = re.compile(r'&\w+&')
    return [
        item for item in data
        if item.get('output') != "" and not any(pattern.search(word) for word in item.get('input', '').split())
    ]

def split_data(data, train_size_ratio=0.8):
    random.shuffle(data)

    train_size = int(len(data) * train_size_ratio)
    train_data = data[:train_size]
    test_data = data[train_size:]

    return train_data, test_data

def save_data(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

def main():
    processed_data = load_data('/content/drive/MyDrive/generation_term_project/processed_data_.json')

    filtered_data = filter_data(processed_data)

    train_data, test_data = split_data(filtered_data)

    save_data(train_data, '/content/drive/MyDrive/generation_term_project/final_train_datasets.json')
    save_data(test_data, '/content/drive/MyDrive/generation_term_project/final_test_datasets.json')

if __name__ == "__main__":
    main()


In [ ]:
import json
from soynlp.tokenizer import RegexTokenizer
from difflib import SequenceMatcher

tokenizer = RegexTokenizer()

def extract_correction_ranges(input_text, corrected_text):
    ranges = []
    matcher = SequenceMatcher(None, input_text, corrected_text)

    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag in {"replace", "delete", "insert"}:
            ranges.append((i1, i2, corrected_text[j1:j2]))
    return ranges

def process_data(input_text, gold_text):

    ranges = extract_correction_ranges(input_text, gold_text)

    ranges_str = ", ".join(f"({start}, {end}, '{replace}')" for start, end, replace in ranges)
    return {"input": input_text, "output": ranges_str}

def process_json(input_json):

    output_data = []

    for document in input_json["document"]:
        for utterance in document["utterance"]:
            input_text = utterance["original_form"]
            gold_text = utterance["corrected_form"]
            processed = process_data(input_text, gold_text)
            output_data.append(processed)

    return output_data

input_file = '/content/drive/MyDrive/generation_term_project/MXEC2102112091.json'  # 예시: 'data.json' (파일 경로를 맞게 설정)
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

output_data = process_json(data)

for entry in output_data:
    print(entry)

output_file = 'processed_data.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, ensure_ascii=False, indent=4)


In [ ]:
import json

def create_dataset(input_json):

    dataset = []

    for document in input_json["document"]:
        for utterance in document["utterance"]:
            original_form = utterance.get("original_form", "")
            corrected_form = utterance.get("corrected_form", "")
            dataset.append({"input": original_form, "output": corrected_form})

    return dataset

input_file = '/content/drive/MyDrive/generation_term_project/MXEC2102112091.json'
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

dataset = create_dataset(data)

for entry in dataset:
    print(entry)

output_file = 'formatted_dataset.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)


In [ ]:
import json
import re

def preprocess_dataset(input_file, output_file):

    with open(input_file, 'r', encoding='utf-8') as f:
        dataset = json.load(f)

    preprocessed_data = []

    name_pattern = re.compile(r'&\w+&')

    for entry in dataset:
        input_text = entry["input"]
        output_text = entry["output"]

        if input_text != output_text and not name_pattern.search(input_text):
            preprocessed_data.append(entry)

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(preprocessed_data, f, ensure_ascii=False, indent=4)

    print(f"전처리 완료: {len(preprocessed_data)}개의 데이터가 저장되었습니다.")

input_file = 'formatted_dataset.json'
output_file = 'preprocessed_dataset.json'

preprocess_dataset(input_file, output_file)


전처리 완료: 291868개의 데이터가 저장되었습니다.


In [ ]:
import json
from difflib import SequenceMatcher

def extract_correction_ranges(input_text, corrected_text):
    ranges = []
    matcher = SequenceMatcher(None, input_text, corrected_text)

    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag in {"replace", "delete", "insert"}:
            ranges.append((i1, i2, corrected_text[j1:j2]))
    return ranges

def process_data(entry):
의 항목을 처리하여 범위와 수정 데이터를 추출
    input_text = entry["input"]
    corrected_text = entry["output"]

    ranges = extract_correction_ranges(input_text, corrected_text)

    ranges_str = ", ".join(f"({start}, {end}, '{replace}')" for start, end, replace in ranges)
    return {"input": input_text, "output": ranges_str}

def process_json(dataset):

    output_data = []
    for entry in dataset:
        processed_entry = process_data(entry)
        output_data.append(processed_entry)
    return output_data

input_file = '/content/preprocessed_dataset.json'
with open(input_file, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

output_data = process_json(dataset)

for entry in output_data:
    print(entry)

output_file = 'propose_processed_data.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, ensure_ascii=False, indent=4)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
{'input': '채소양념이랑 만들어먹으면 맛날거같아', 'output': "(2, 2, ' '), (10, 10, ' '), (16, 16, ' '), (17, 17, ' '), (19, 19, '.')"}
{'input': '아', 'output': "(1, 1, '.')"}
{'input': '시래기 때문에 맛나기보다는 그냥 양념이 맛난건가??', 'output': "(24, 24, ' '), (27, 28, '')"}
{'input': '개인적으로 나는 그랬어 ㅋㅋ', 'output': "(12, 12, '.')"}
{'input': '근데 좀 짜서그런가', 'output': "(7, 7, ' ')"}
{'input': '나는 딱좋았는데', 'output': "(4, 4, ' '), (8, 8, '.')"}
{'input': '먹다가 조금 질릴뻔했어', 'output': "(9, 9, ' '), (12, 12, '.')"}
{'input': '아진짜로?', 'output': "(1, 1, ', ')"}
{'input': '물하고 탄산을 자꾸 찾게되더라 ㅋㅋ', 'output': "(13, 13, ' '), (16, 16, '.')"}
{'input': '마지막 오뎅까지 먹었으면서....', 'output': "(17, 18, '')"}
{'input': '그 쌈 아직도 생각나', 'output': "(11, 11, '.')"}
{'input': '같이나온 고기가 좀 짠거같았어', 'output': "(2, 2, ' '), (12, 12, ' '), (13, 13, ' '), (16, 16, '.')"}
{'input': '맛집인거같기는해', 'output': "(3, 3, ' '), (4, 4, ' '), (7, 7, ' '), (8, 8, '.')"}
{'input': '김밥을 상추에 싸먹은건 처음인데', 'output': "(11, 11, ' ')"}
{'input': '자꾸 자꾸 먹